# Gaussian 

Modified: Oct 12, 2019

This is a small note on how I think of Gaussian distribution, $N(\mu, \sigma)$

## Load libraries

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import pandas as pd
    
from pathlib import Path #we'll import Path object with `.ls` method added later
from pprint import pprint as pp

import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts


# import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

# Dashboards
import param as pm, panel as pn
pn.extension()

In [ ]:
# Geoviews visualization default options
H,W, = 250,250
opts.defaults(
    opts.RGB(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom']),
    opts.Image(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom'], framewise=True),#axiswise=True ),
    opts.Points( tools=['hover'], active_tools=['wheel_zoom']),
    opts.Curve( tools=['hover'], active_tools=['wheel_zoom'], padding=0.1),

)

## Set up additional library path

In [ ]:
# Add the utils directory to the search path
SP_ROOT = Path.home()/'Playground/ContextNet'
SP_LIBS = SP_ROOT/'scripts' # to be changed to 'src'
# LIBS_DIR = Path('../src').absolute()
DIRS_TO_ADD = [SP_LIBS]#, LIBS_DIR]
for p in DIRS_TO_ADD:
    assert p.exists()
    
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))
        print(f"Added to sys.path: {p}")

# pp(sys.path)
    

In [ ]:
from output_helpers import print_mro as mro, nprint, Path
import SpacenetPath as spp
import spacenet_globals as spg
# from output_helpers import Path #.ls method is added to Path class

In [ ]:
import osmnx as ox
import rasterio as rio
from rasterio.plot import reshape_as_image
import skimage as ski

## 1. Intuition

The probability density function, $f(x \mid \mu, \sigma)$, of the normal distribution $N(\mu, \sigma)$ is:

$$
f(x \mid \mu, \sigma) = \frac{1}{Z} \text{exp} [-\frac{1}{2}(\frac{x-\mu}{\sigma})^{2}]
$$

where $Z = \sqrt{2\pi\sigma^{2}}$ is the normalization factor that makes $f$ a distribution that sums up to $1$.

In this form, the purpose of $\sigma$ and $\mu$ becomes more explicit. 
1. $(x-mu)$: we view a difference (vector) between $\mu$ and $x$. Let $d := x-\mu$
2. $ \frac{\lVert d \rVert}{\sigma}$: How big is the size of this difference vector in $\sigma$ units?
That is, how many $\sigma$-sized steps do we need to take to get to $x$ from $\mu$?


Now, let's say, we have "normalized" the length of the difference vector (between $\mu$ and a given $x$) by $\sigma$. 
Let's call this "normalized distance" as $d_{\text{normed}}$. Then, $f$ can be expressed as:

$$
f(x \mid \mu, \sigma) = f(d_{\text{normed}}) \propto \text{exp} [-\frac{1}{2} d_{\text{normed}}^{2}]
$$

<img src="assets/gaussian.png" alt="gaussian" width="800"/>


## 2. Visualization 

Let's see what this function looks like in 2Dim and 3Dim space. 

- First in 2Dim

$f(d) = exp[-\frac{1}{2} d^2]$

Or, 

$$f(d) = exp[-\frac{1}{2} d^2]$$
where the difference vector, $\vec{d}$, is:

$$\vec{d} = \vec{x} - \vec{\mu}$$ 

and the normalized length of the difference vector, $d_\text{norm}$, is:
$$d_\text{norm} = \frac{\lVert \vec{d}\rVert} {\sigma}$$

Using holoviews for visualization...

In [ ]:
def draw_gaussian_2d(xmin=-5, xmax=5, n_points=100):
    xs = np.linspace(xmin, xmax, num=n_points)
    ys = np.exp(-0.5*xs**2)
    overlay = (
        hv.Curve((xs,ys)) 
        * hv.VLine(0).opts(color='black', line_width=1)
        * hv.VLine(-4).opts(color='red', line_width=1, line_dash='dashed')
        * hv.VLine(4).opts(color='red', line_width=1, line_dash='dashed')
    )
    return overlay

In [ ]:
%%opts Curve [width=600, show_grid=True] 
draw_gaussian_2d()

- 3Dim
$$
f(\mathbf{x} \mid \mathbf{\mu}, \Sigma)= \frac{1}{Z} \text{exp}[-\frac{1}{2} (\mathbf{x}-\mathbf{\mu} )^{T} \Sigma^{-1} (\mathbf{x}-\mathbf{\mu})]
$$

where 

$$
Z = \sqrt{(2\pi)^{m} \det(\Sigma)}
$$
for $\mathbf{x} \in R^{m}$

In [ ]:
def multivariate_gaussian(xs, mu, cov):
    """
    Vectorized version of multivariate_gaussian pmf
    This has a computational advantage of computing determinate of the covariance 
    matrix only once for multiple x points, in addition to a vectorized linalg.solve
    operation
    
    Args:
    - xs (np.array): a 2Dim array whose column is a vector of indivisual x in R^m 
    to compute the gaussian. For example, if x is a two dimensional vector, 
    `xs.shape` should be (2, num_xs)
    
    - mu (np.array): 1 dim array for a mu vector
    - cov (np.array): m by m matrix where m = dimension of x (ie. xs.shape[0])
    """
    assert xs.ndim == 2, "xs must be a two dimensional collection of column vectors"
    assert mu.ndim == 2, "mu must be two dimensional"
    assert mu.shape[1] == 1, "mu must be a column vector, ie.shape of (m,1) where m is the data dimension"

    det = np.linalg.det(cov)
    xs_m = xs - mu
#     print(xs-mu)
#     print(xs_m)
    Z = np.sqrt(2*np.pi*det)
#     energy = (x_m.T).dot(np.linalg.inv(cov)).dot(x_m)
    xsTcinv =  (np.linalg.solve(cov, xs_m)).T
#     print('xsTcinv shape: ', xsTcinv.shape)
    xs_m = xs_m.T
#     print('xs_m transposed: ', xs_m.shape)
    energies = np.array([xTcinv.dot(x_m) for (xTcinv,x_m) in zip(xsTcinv, xs_m)])
    return np.exp(-.5*energies)/Z
    

In [ ]:
#Simple tests
# x = np.array([0,0]).reshape((-1,1))
x = np.array([[0,0], [1,0]]).T

mu = np.array([0,0]).reshape((-1,1)) #column vector
cov = np.array([[1,0],[0,1]])
multivariate_gaussian(x, mu, cov)

In [ ]:
def draw_gaussian_3d(xmin, xmax, nx, ymin, ymax, ny, mu, cov):
    xx, yy = np.meshgrid(np.linspace(xmin, xmax, nx), np.linspace(ymin, ymax, ny))
    points = np.array([ list(xx.flat), list(yy.flat)]) # each column contains a point(ie. (x1,x2))
    heights = multivariate_gaussian(points, mu, cov)
    surface = hv.Surface(heights.reshape((nx, ny)), bounds=(xmin, ymin, xmax, ymax)) #slow
#     surface = hv.Surface((xx.flat, yy.flat, heights)) #slow

#     surface = hv.TriSurface((xx.flat, yy.flat, heights)) #with plotly backend

    
    return surface

In [ ]:
hv.extension('plotly') # currently bokeh backend doesn't support 3D plot with holoviews objects

xmin, xmax, nx = -5, 5, 50
ymin, ymax, ny = -5, 5, 50

mu = np.array([0,0]).reshape((-1,1)) #column vector with two dims
cov0 = np.array([[1,0],[0,1]])
cov1 = np.array([[1,0], [0,2]])
cov2 = np.array([[2,0],[0,2]])

surf0 = draw_gaussian_3d(xmin, xmax, nx, ymin, ymax, ny, mu, cov0)
surf1 = draw_gaussian_3d(xmin, xmax, nx, ymin, ymax, ny, mu, cov1)
surf2 = draw_gaussian_3d(xmin, xmax, nx, ymin, ymax, ny, mu, cov2)

In [ ]:
%%opts TriSurface [colorbar=True, width=400, height=400] (alpha=0.5) Surface [colorbar=True, width=400, height=400] (alpha=0.5)

(
    surf0.relabel(group='cov:[[1,0],[0,1]]').opts(cmap='Blues')
    * surf1.relabel(group='cov:[[1,0],[0,2]]').opts(cmap='Reds')
    * surf2.relabel(group='cov:[[2,0],[0,2]]')
)

In [ ]:
def draw_g3d(mu_x, mu_y, sig_x, sig_y, sig_xy):
    mu = np.array([mu_x, mu_y]).reshape((-1,1))
    cov = np.array([[sig_x**2,sig_xy],
                    [sig_xy, sig_y**2]])
    return draw_gaussian_3d(xmin, xmax, nx,
                            ymin, ymax, ny,
                            mu, cov)

In [ ]:
dmap_3d = hv.DynamicMap(draw_g3d, kdims=['mu_x', 'mu_y', 'sig_x', 'sig_y', 'sig_xy'])


In [ ]:
%%opts Surface [colorbar=True, width=600, height=600] (alpha=0.9)

dmap_3d.redim.values(mu_x=range(-2,2),
                     mu_y=range(-2,2), 
                     sig_x=np.linspace(0.3, 3.0, 10),
                     sig_y=np.linspace(0.3, 3.0, 10), 
                     sig_xy=[0.])

Demo of my 2Dim gaussian animation: [here](http://recordit.co/2D2kJ8OM6T)

## Application: Orientation-Energy based contour detection 
### Elongated Gaussian 

New parameter $\alpha$ is introduced to relate $\sigma_{x}$ and $\sigma_{y}$ s.t. $\sigma_{x} = \alpha \sigma_{y}$.  
We will only consider an axis-aligned gaussian distribution(ie. $\sigma_{xy} = \sigma_{ys} = 0$, or equivalently, $\Sigma$ is a diagonal matrix)


In [ ]:
def draw_elongated_gauss_2d(mu_x, mu_y, sig_y, alpha):
    """
    mu_x (float)
    mu_y (float)
    sig_y (float): standard deviation in Y
    alpha (float): controls the ratio between sig_x and sig_y st. sigma_x = alpha * sigma_y
    """
    
    mu = np.array([mu_x, mu_y]).reshape((-1,1))
    sig_x = alpha * sig_y
    cov = np.array([[sig_x**2, 0],
                    [0, sig_y**2]])
    return draw_gaussian_3d(xmin, xmax, nx,
                            ymin, ymax, ny,
                            mu, cov)

In [ ]:
dmap_elong = hv.DynamicMap(draw_elongated_gauss_2d, 
                           kdims=['mu_x', 'mu_y', 'sig_y', 'alpha']
                          )

In [ ]:
%%opts Surface [colorbar=True, width=600, height=600] (alpha=0.9)

dmap_elong.redim.values(mu_x=range(-2,2),
                        mu_y=range(-2,2), 
                        sig_y=[0.3, 1.0, 1.5, 3.0, 5.0], 
                        alpha=np.linspace(0.5, 2.0, 10))

### Hilbert transform
- [Richard Gao's post](https://tinyurl.com/y4grv4ns)
- [Application in Medical Imaging](https://tinyurl.com/y4grv4ns)
- [Useful resources to read](https://tinyurl.com/y3dcd5tq)
- [Paper: An Approach to the 2D Hilbert Transform for Image Processing Applications
](https://tinyurl.com/yx8wak3s)
- Pyhht [doc](https://readthedocs.org/projects/pyhht/downloads/pdf/dev/)

In [ ]:
import scipy.signal as ss
from scipy import fftpack

### Fourier Transform review
- [scipy tutorial on denoising using dft](https://tinyurl.com/y2m7t9ze)
- [opencv tutorial](https://tinyurl.com/y2ho27sl)
- [allsignalprocessing tutorial](https://tinyurl.com/y3rtznv2)
- Porf. Fred Hamprecht's [courses](https://tinyurl.com/y4dkk8s8): He has other awesome lectures such as Rattern Recognition, ML for CV, Image Analysis. He uses OneNote for his presentations:)
- [Spatial and Frequency domains - Roger Bourne](https://tinyurl.com/y2xyvscr)

In [ ]:
import skimage as ski

In [ ]:
coin = (ski.data.coins()/255.).astype(np.float32)
plt.imshow(coin)

In [ ]:
coin_fft = fftpack.fft2(coin)

In [ ]:
# Show the results
# A logarithmic colormap
from matplotlib.colors import LogNorm

def plot_spectrum(im_fft):
    plt.imshow(np.abs(im_fft), norm=LogNorm(vmin=5), cmap='gray')
    plt.colorbar()

plt.figure()
# plot_spectrum(coin_fft)
plot_spectrum(np.fft.fftshift(coin_fft))

plt.title('Fourier transform')

In [ ]:
import cv2

In [ ]:
cv_dft = cv2.dft(coin, flags=cv2.DFT_COMPLEX_OUTPUT)
dft_shift= np.fft.fftshift(cv_dft)

In [ ]:
mag_spectrum = cv2.magnitude(dft_shift[:,:,0], dft_shift[:,:,1])

In [ ]:
plt.imshow(mag_spectrum, norm=LogNorm(vmin=5), cmap='gray')
plt.colorbar();

Why do we want to shift (`np.fft.fftshift`) the magnitude plot of `np.fft` result?
![](../assets/2d-dft-mag-shifted.png)

---
Back to Hilbert transform
### 2Dim Hilbert transform
Conventional implementation by Marple


In [ ]:
def 

---
## Test Images for Signal Processing

Visual grating patterns are good test images for signal processing algorithms.  They are often used in psychology/cogsci experiements. I found some useful libraries that helps generating these patterns in Python.

- [ImaGen](https://tinyurl.com/yyx2dt3l): Developed by HoloViz team <3. Recommended.
- [psychology](https://tinyurl.com/y36crpee)

In [ ]:
import imagen as ig


In [ ]:
np.set_printoptions(precision=4)

In [ ]:
line_gen = ig.Line(xdensity=5, ydensity=5, smoothing=0)
l1 = line_gen().astype(np.float32)
l2 = line_gen(smoothing=0.1,orientation=0.8,thickness=0.4).astype(np.float32)

# Gabor pattern (256x256)
gabor_gen = ig.Gabor()
gabor = gabor_gen().astype(np.float32)

In [ ]:
from functools import partial

In [ ]:
cvdft = partial(cv2.dft, flags=cv2.DFT_COMPLEX_OUTPUT)
dft = cvdft(l1) # channel_0 encodes real part, channel_1 encodes complex part
dft_shift = np.fft.fftshift(dft)

mag = cv2.magnitude(dft_shift[:,:,0], dft_shift[:,:,1])

In [ ]:
%opts Image [colorbar=True]

In [ ]:
hv.Image(l1) + hv.Image(np.log(mag+1))

In [ ]:
gabor.dtype

In [ ]:
def get_dft_mag(im32):
    """
    im32: two dimensional (ie. grayscale) np.float32 dtype np.array
    
    Returns:
    
    """
    assert im32.dtype == np.float32, "input image must be grayscale, np.float32" # todo better checking
    dft = cvdft(im32)
    dft_shift = np.fft.fftshift(dft)
    mag = cv2.magnitude(dft_shift[:,:,0], dft_shift[:,:,1])
    return mag

def show_dft_mag(im32, log_scale=True):
    mag = get_dft_mag(im32)
    if log_scale:
        mag = np.log2(mag+1)
        
        
    return hv.Image(mag)


 

In [ ]:
hv.extension('bokeh')   

In [ ]:
%%opts Image [shared_axes=False, colorbar=True] {+framewise}
(
    hv.Image(gabor).opts(axiswise=True) + show_dft_mag(gabor)  + show_dft_mag(gabor, log_scale=False)
)

In [ ]:
type(l)

In [ ]:
# hv.help(hv.Image)

In [ ]:
def

---
Other helpful resources:
- [Multivariate Gaussian](https://peterroelants.github.io/posts/multivariate-normal-primer/)
- [Wiki](#)